In [1]:
from ctypes import *
import time
import numpy as np
import matplotlib.pyplot as plt
import re

In [37]:
class PythonedDLL(object):
    def __init__(self, lib_filename, header_filename):
        self.lib = cdll.LoadLibrary(lib_filename)
        
        consts = {}
        errs = {}
        const_re = re.compile(r"""((?:DRV|AC|SHAMROCK)_[A-Z_]+?) ([0-9]+)""")
        with open(header_filename) as f:
            for line in f:
                found_groups = const_re.findall(line)
                if found_groups:
                    name, num = found_groups[0]
                    consts[name] = int(num)
                    if len(num) == 5:
                        errs[int(num)] = name
        
        self.consts = consts
        self.errs = errs
        
        self.already_decorated_funcs = {}
        
        self.int_ = byref(c_int())
        self.long_ = byref(c_long())
        
    def __getattribute__(self, name):
        if name[0].islower():
            return super(PythonedDLL, self).__getattribute__(name)
        elif name in self.already_decorated_funcs:
            return self.already_decorated_funcs[name]
        else:
            new_func = self.lib_decorator(self.lib.__getattr__(name))
            self.already_decorated_funcs[name] = new_func
            return new_func
        
    def lib_decorator(self, func):
        def new_func(*args):
            msg = self.errs[func(*args)]
            if "SUCCESS" in msg:
                print msg
            else:
                raise IOException(msg)
            
            # return value: tuple of values of pointers passed as arguments
            # if just one pointer, return just its value
            # if no pointers, return None
            ret = []
            for arg in args:
                try:
                    ret.append(arg._obj.value)
                except AttributeError: pass
            if len(ret) == 0:
                return
            elif len(ret) == 1:
                return ret[0]
            else:
                return tuple(ret)
        return new_func    

"""
with open("ATMCD32D.H") as f:
    atmcd32d_header = f.read()
with open("ShamrockCIF.h") as f:
    shamrockcif_header = f.read()

consts = re.findall(r"((?:DRV|AC|SHAMROCK)_[A-Z_]+?) ([0-9]+?)\n",
           atmcd32d_header + shamrockcif_header)
err_dict = {int(num): name for name, num in consts if len(num) == 5}
print err_dict

cam_lib = cdll.LoadLibrary("atmcd64d.dll")
spec_lib = cdll.LoadLibrary("ShamrockCIF.dll")
"""

None

In [38]:
cam_lib = PythonedDLL("atmcd64d.dll", "ATMCD32D.H")

In [39]:
spec_lib = PythonedDLL("ShamrockCIF.dll", "ShamrockCIF.h")

In [40]:
spec_lib.ShamrockInitialize(".")

SHAMROCK_SUCCESS


In [41]:
spec_lib.ShamrockGetNumberDevices(c_int_ptr)

SHAMROCK_SUCCESS


1

In [42]:
spec_lib.ShamrockFlipperMirrorIsPresent(0, 2, c_int_ptr)

SHAMROCK_SUCCESS


1

In [43]:
spec_lib.ShamrockGetFlipperMirror(0, 2, c_int_ptr)

SHAMROCK_SUCCESS


0

In [44]:
spec_lib.ShamrockSetFlipperMirror(0, 2, 0)

SHAMROCK_SUCCESS


In [11]:
class AndorCamera(object):
    name_dict = {17910: "newton",
                 17911: "idus"}
    
    def check_current(self):
        cur_camera_handle = c_int()
        cam_lib.GetCurrentCamera(byref(cur_camera_handle))
        if not cur_camera_handle == self.handle:
            raise IOException("%s is not current camera" % self.name)
            
    def __init__(self, handle, name=None):
        self.handle = handle
        self.check_current()
        
        self.serial_no = cam_lib.GetCameraSerialNumber(cam_lib.int_)
        try: 
            raise IOException("Unknown name: camera with serial no %d" % self.serial_no)
        
cams = {}
for i in xrange(cam_lib.GetAvailableCameras(cam_lib.int_)):
    cam = AndorCamera("
        

n_cams = c_int()
print cam_lib.GetAvailableCameras(byref(n_cams))
print "Number of cams: %d" n_cams

newton_handle = c_int()
idus_handle = c_int()
print cam_lib.GetCameraHandle(0, byref(newton_handle))
print "First camera handle: %d" % newton_handle
print cam_lib.GetCameraHandle(1, byref(idus_handle))
print "Second camera handle: %d" % idus_handle

newton_ser_num = c_int()
idus_ser_num = c_int()
print cam_lib.SetCurrentCamera(newton_handle)
print cam_lib.GetCameraSerialNumber(byref(newton_ser_num))
print "First camera serial number: %d" % newton_ser_num
print cam_lib.SetCurrentCamera(idus_handle)
print cam_lib.GetCameraSerialNumber(byref(idus_ser_num))
print "Second camera serial number: %d" % idus_ser_num

IndentationError: expected an indented block (<ipython-input-11-709c8fa6c5d6>, line 3)